In [1]:
from pathlib import Path
import datetime as dt
import pandas as pd

In [2]:
# Create a function to save the DataFrames to CSV files
def save_csv_file(df, file_path):
    """ 
    Save a DataFrame to a CSV file at the specified file path.

    Parameters:
    - df: DataFrame to save
    - file_path: Path to save the CSV file
    """
    
    # Check if the parent directory exists
    if not file_path.parent.exists():
        print(f"Error: The directory `{file_path.parent}` does not exist.")
        return
    
    if file_path.exists():
        print(f"File `{file_path.name}` already exists. Overwriting file.")
        file_path.unlink()
    
    # Save the DataFrame to the specified file path
    df.to_csv(file_path, index=False)
    print(f"File saved as `{file_path.name}`")

In [3]:
# Load the main California DataFrame that we will be merging with the income data
ca_df = pd.read_csv(
    Path("../../../../../data/processed_data/ca_main_dataset.csv"),
)

print("Shape:", ca_df.shape)
ca_df

Shape: (600, 7)


,year,state,county,zip_codes,population,ev_registrations,cumulative_ev_stations
0,2010,CA,Alameda County,"[94501, 94502, 94536, 94537, 94538, 94539, 945...",1512986.0,20,0
1,2010,CA,Amador County,"[95601, 95629, 95640, 95642, 95654, 95665, 956...",37886.0,1,1
2,2010,CA,Contra Costa County,"[94505, 94506, 94507, 94509, 94511, 94513, 945...",1052540.0,10,0
3,2010,CA,Fresno County,"[93210, 93234, 93242, 93602, 93605, 93606, 936...",932039.0,2,0
4,2010,CA,Humboldt County,"[95501, 95502, 95503, 95511, 95514, 95518, 955...",135009.0,2,0
...,...,...,...,...,...,...,...
595,2020,CA,Tuolumne County,"[95305, 95309, 95310, 95314, 95321, 95327, 953...",55379.0,240,9
596,2020,CA,Ventura County,"[91319, 91320, 91358, 91360, 91361, 91362, 913...",843371.0,13080,117
597,2020,CA,Yolo County,"[95605, 95606, 95607, 95612, 95616, 95617, 956...",216291.0,2948,53
598,2020,CA,Yuba County,"[95692, 95901, 95903, 95918, 95919, 95922, 959...",81958.0,289,3


In [4]:
# Load the California Income DataFrame
ca_income_df = pd.read_csv(
    Path("../../../../../data/processed_data/ca_county_income.csv"),
)

# Display the DataFrame
print("Shape:", ca_income_df.shape)
ca_income_df.head()

Shape: (58, 12)


,County,2010 Median Income,2011 Median Income,2012 Median Income,2013 Median Income,2014 Median Income,2015 Median Income,2016 Median Income,2017 Median Income,2018 Median Income,2019 Median Income,2020 Median Income
0,Alameda County,"69,384","70,821","71,516","72,112","73,775","75,619","79,831","85,743","92,574","99,406","104,888"
1,Alpine County,"63,478","59,018","59,931","58,636","61,343","52,917","62,375","63,438","64,688","63,750","85,750"
2,Amador County,"54,758","56,180","53,462","53,684","52,964","54,171","57,032","60,636","61,198","62,772","65,187"
3,Butte County,"43,170","42,971","43,339","43,752","43,165","43,444","44,366","46,516","48,443","52,537","54,972"
4,Calaveras County,"54,971","55,256","54,686","55,295","54,936","53,233","53,502","54,800","58,151","63,158","67,054"


In [5]:
# Inspect both County Values in both DataFrames
# To ensure that the county names are the same in both DataFrames, we will compare the unique county values in each DataFrame.
main_df_counties = set(ca_df['county'])
income_df_counties = set(ca_income_df['County']) 

# Find the counties that don't match
mismatches = main_df_counties.symmetric_difference(income_df_counties)

# Show mismatches
print("Counties that don't match:", mismatches)

Counties that don't match: {nan}


In [6]:
# Reshape the DataFrame 'ca_income_df' from wide format to long format using the melt function
# Param 1: The DataFrame to be reshaped
# Param 2: Column(s) to use as identifier variables (these columns will remain as-is)
# Param 3: Name of the new column that will contain the original column names (years in this case)
# Param 4: Name of the new column that will contain the values from the original columns
ca_income_df = pd.melt(
    ca_income_df, 
    id_vars=["County"], 
    var_name="year", 
    value_name="median_income", 
)

# Display the DataFrame 
print("Shape:", ca_income_df.shape)
ca_income_df.head()

Shape: (638, 3)


,County,year,median_income
0,Alameda County,2010 Median Income,"69,384"
1,Alpine County,2010 Median Income,"63,478"
2,Amador County,2010 Median Income,"54,758"
3,Butte County,2010 Median Income,"43,170"
4,Calaveras County,2010 Median Income,"54,971"


In [7]:
# From the 'year' column, extract only the year values using the 'str.extract' function
# The regular expression '\d{4}' will match any four consecutive digits
ca_income_df['year'] = ca_income_df['year'].str.extract('(\d{4})')

# Confirm that the 'year' column only contains year values
print("Shape:", ca_income_df.shape)
ca_income_df

Shape: (638, 3)


,County,year,median_income
0,Alameda County,2010,"69,384"
1,Alpine County,2010,"63,478"
2,Amador County,2010,"54,758"
3,Butte County,2010,"43,170"
4,Calaveras County,2010,"54,971"
...,...,...,...
633,Tulare County,2020,"52,534"
634,Tuolumne County,2020,"60,509"
635,Ventura County,2020,"89,295"
636,Yolo County,2020,"73,746"


In [8]:
# Inspect the data types of the columns in both DataFrames
ca_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   County         638 non-null    object
 1   year           638 non-null    object
 2   median_income  638 non-null    object
dtypes: object(3)
memory usage: 15.1+ KB


In [9]:
# Convert the 'year' column to an integer data type
ca_income_df["year"] = ca_income_df["year"].astype(int)

ca_income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   County         638 non-null    object
 1   year           638 non-null    int32 
 2   median_income  638 non-null    object
dtypes: int32(1), object(2)
memory usage: 12.6+ KB


In [10]:
# Merge the main California DataFrame with the reshaped California Income DataFrame
ca_df_clean = pd.merge(ca_df, ca_income_df, left_on=['county', 'year'], right_on=['County', 'year'], how='left')

# Remove the redundant 'County' column
ca_df_clean.drop(columns=['County'], inplace=True)

print("Shape:", ca_df_clean.shape)
ca_df_clean.head()

Shape: (600, 8)


,year,state,county,zip_codes,population,ev_registrations,cumulative_ev_stations,median_income
0,2010,CA,Alameda County,"[94501, 94502, 94536, 94537, 94538, 94539, 945...",1512986.0,20,0,"69,384"
1,2010,CA,Amador County,"[95601, 95629, 95640, 95642, 95654, 95665, 956...",37886.0,1,1,"54,758"
2,2010,CA,Contra Costa County,"[94505, 94506, 94507, 94509, 94511, 94513, 945...",1052540.0,10,0,"78,385"
3,2010,CA,Fresno County,"[93210, 93234, 93242, 93602, 93605, 93606, 936...",932039.0,2,0,"46,430"
4,2010,CA,Humboldt County,"[95501, 95502, 95503, 95511, 95514, 95518, 955...",135009.0,2,0,"40,089"


In [11]:
# Inspect for missing values
ca_df_clean.isnull().sum()

year                       0
state                      0
county                    11
zip_codes                 11
population                11
ev_registrations           0
cumulative_ev_stations     0
median_income             11
dtype: int64

In [12]:
# Save the cleaned California DataFrame to a CSV file
file_name = f"ca_ev_main_dataset.csv"
file_path = Path(f"../../../../../data/processed_data/{file_name}")
save_csv_file(ca_df_clean, file_path)

File saved as `ca_ev_main_dataset.csv`
